In [1]:
# Imports for Dataset class
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

# Imports for dataloader
from torch.utils.data import DataLoader

# Imports for CNN
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

# Imports for plotting
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [3]:
class ASLDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                                            on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_mapping = {}  # Mapping from original labels to continuous range

        # Collect all image paths and labels
        for filename in os.listdir(root_dir):
            if filename.startswith('color_'):
                # Extract label from filename
                parts = filename.split('_')
                label = int(parts[1])  # Assuming the label is the second part of the filename
                self.image_paths.append(os.path.join(root_dir, filename))
                self.labels.append(label)

        # Create a continuous label mapping
        unique_labels = sorted(set(self.labels))
        self.label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
        self.labels = [self.label_mapping[label] for label in self.labels]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Ensure image is in grayscale
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


root_dir = 'dataset5_50x50_split/'
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1] range for grayscale
])

# Create dataset instances
train_dataset = ASLDataset(root_dir=os.path.join(root_dir, 'train'), transform=transform)
val_dataset = ASLDataset(root_dir=os.path.join(root_dir, 'val'), transform=transform)
test_dataset = ASLDataset(root_dir=os.path.join(root_dir, 'test'), transform=transform)


In [4]:
# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
# Class for CNN following paper 1 architecture
class PaperCNN(nn.Module):
    def __init__(self, num_classes=24, p_dropout = 0.3):
        super(PaperCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.dropout1 = nn.Dropout2d(p_dropout)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.dropout2 = nn.Dropout2d(p_dropout)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(128, 512, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        self.dropout3 = nn.Dropout2d(0.3)
        
        self.conv4 = nn.Conv2d(512, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.dropout4 = nn.Dropout2d(p_dropout)
        
        self.conv5 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.dropout5 = nn.Dropout2d(p_dropout)
        
        self.fc1 = nn.Linear(32 * 3 * 3, 512)
        self.dropout_fc1 = nn.Dropout(p_dropout)
        
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        # convrelu - dropout - bn
        x = self.bn1(self.dropout1(F.relu(self.conv1(x))))
#         print(f"Shape after conv1 + relu + dropout + bn1: {x.shape}")
        
        x = self.bn2(self.dropout2(F.relu(self.conv2(x))))
#         print(f"Shape after conv2 + relu + dropout + bn2: {x.shape}")
        x = self.pool(x)
#         print(f"Shape after pool: {x.shape}")
        
        x = self.bn3(self.dropout3(F.relu(self.conv3(x))))
#         print(f"Shape after conv3 + relu + dropout + bn3: {x.shape}")
        x = self.pool(x)
#         print(f"Shape after pool: {x.shape}")
        
        x = self.bn4(self.dropout4(F.relu(self.conv4(x))))
#         print(f"Shape after conv4 + relu + dropout + bn4: {x.shape}")
        x = self.pool(x)
#         print(f"Shape after pool: {x.shape}")
        
        x = self.bn5(self.dropout5(F.relu(self.conv5(x))))
#         print(f"Shape after conv5 + relu + dropout + bn4: {x.shape}")
        x = self.pool(x)
#         print(f"Shape after pool: {x.shape}")
        
        x = x.view(-1, 32 * 3 * 3)  # Flatten the output of the last conv layer
#         print(f"Shape after view: {x.shape}")
        
        x = self.dropout_fc1(F.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return x

In [6]:
# Function to infer number of classes
def get_num_classes(dataset):
    """
    Get the number of unique classes in the dataset.

    Args:
        dataset (Dataset): The dataset to analyze.

    Returns:
        int: The number of unique classes.
    """
    unique_labels = set()
    for _, label in dataset:
        unique_labels.add(label)
    return len(unique_labels)

In [ ]:
# Initialize the model, loss function, and optimizer

p_dropouts = [0.1, 0.2, 0.3, 0.4, 0.5]
lrs = [0.0001, 0.0005, 0.001, 0.005, 0.01]
for p_dropout in p_dropouts:
    for lr in lrs: 
        num_classes = get_num_classes(train_dataset)
        model = PaperCNN(num_classes=num_classes, p_dropout = p_dropout)
        device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
        model.to(device)
        #print("Model initialized")
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        #print("Optimizer initialized")
        
        # Initialize lists to track metrics
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []
        
        
        # Training loop with tqdm progress bar
        num_epochs = 10
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            correct_train = 0
            total_train = 0
            train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
            for images, labels in train_loader_tqdm:
                # move images and labels to device
                images, labels = images.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                train_loader_tqdm.set_postfix(loss=running_loss/len(train_loader))
                
                # Calculate training accuracy
                _, predicted = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
        
            epoch_loss = running_loss / len(train_loader)
            train_losses.append(epoch_loss)
            train_accuracy = 100 * correct_train / total_train
            train_accuracies.append(train_accuracy)
            
            #print(f"Epoch {epoch+1}, Loss: {epoch_loss}, Accuracy: {train_accuracy}%")
        
            # Validation loop
            model.eval()
            running_val_loss = 0.0
            correct_val = 0
            total_val = 0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for images, labels in val_loader:
                    # move images and labels to device
                    images, labels = images.to(device), labels.to(device)
                    
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    running_val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).sum().item()
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())
            
            epoch_val_loss = running_val_loss / len(val_loader)
            val_losses.append(epoch_val_loss)
            val_accuracy = 100 * correct_val / total_val
            val_accuracies.append(val_accuracy)
        final_train_loss = train_losses[-1]
        final_val_loss = val_losses[-1]
        final_train_accuracy = train_accuracies[-1]
        final_val_accuracy = val_accuracies[-1]
        print(f"Dropout: {p_dropout}, LR: {lr}, Validation Loss: {final_val_loss}, Validation Accuracy: {final_val_accuracy}%")
        
        
        print("Training complete")

Epoch 10/10: 100%|██████████| 1439/1439 [03:06<00:00,  7.72batch/s, loss=0.0312]


Dropout: 0.1, LR: 0.0001, Validation Loss: 0.01621346259610215, Validation Accuracy: 99.44252990066896%
Training complete


Epoch 10/10: 100%|██████████| 1439/1439 [03:26<00:00,  6.96batch/s, loss=0.0221]


Dropout: 0.1, LR: 0.0005, Validation Loss: 0.017602893635198538, Validation Accuracy: 99.49320900060815%
Training complete


Epoch 10/10: 100%|██████████| 1439/1439 [03:37<00:00,  6.62batch/s, loss=0.0296]


...

Epoch 10/10: 100%|██████████| 1439/1439 [02:57<00:00,  8.13batch/s, loss=0.157]


Dropout: 0.4, LR: 0.0005, Validation Loss: 0.03115209849812407, Validation Accuracy: 99.13845530103386%
Training complete


Epoch 10/10: 100%|██████████| 1439/1439 [03:09<00:00,  7.60batch/s, loss=0.174]


Dropout: 0.4, LR: 0.001, Validation Loss: 0.03141675984771759, Validation Accuracy: 99.13845530103386%
Training complete


Epoch 8/10:  78%|███████▊  | 1124/1439 [03:05<01:34,  3.32batch/s, loss=0.241]

In [7]:
# Initialize the model, loss function, and optimizer

p_dropouts = [0.4, 0.5]
lrs = [0.0001, 0.0005, 0.001, 0.005, 0.01]
for p_dropout in p_dropouts:
    for lr in lrs: 
        num_classes = get_num_classes(train_dataset)
        model = PaperCNN(num_classes=num_classes, p_dropout = p_dropout)
        device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
        model.to(device)
        #print("Model initialized")
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        #print("Optimizer initialized")
        
        # Initialize lists to track metrics
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []
        
        
        # Training loop with tqdm progress bar
        num_epochs = 10
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            correct_train = 0
            total_train = 0
            train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
            for images, labels in train_loader_tqdm:
                # move images and labels to device
                images, labels = images.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                train_loader_tqdm.set_postfix(loss=running_loss/len(train_loader))
                
                # Calculate training accuracy
                _, predicted = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
        
            epoch_loss = running_loss / len(train_loader)
            train_losses.append(epoch_loss)
            train_accuracy = 100 * correct_train / total_train
            train_accuracies.append(train_accuracy)
            
            #print(f"Epoch {epoch+1}, Loss: {epoch_loss}, Accuracy: {train_accuracy}%")
        
            # Validation loop
            model.eval()
            running_val_loss = 0.0
            correct_val = 0
            total_val = 0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for images, labels in val_loader:
                    # move images and labels to device
                    images, labels = images.to(device), labels.to(device)
                    
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    running_val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).sum().item()
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())
            
            epoch_val_loss = running_val_loss / len(val_loader)
            val_losses.append(epoch_val_loss)
            val_accuracy = 100 * correct_val / total_val
            val_accuracies.append(val_accuracy)
        final_train_loss = train_losses[-1]
        final_val_loss = val_losses[-1]
        final_train_accuracy = train_accuracies[-1]
        final_val_accuracy = val_accuracies[-1]
        print(f"Dropout: {p_dropout}, LR: {lr}, Validation Loss: {final_val_loss}, Validation Accuracy: {final_val_accuracy}%")
        
        
        print("Training complete")

Epoch 10/10: 100%|██████████| 1439/1439 [03:42<00:00,  6.46batch/s, loss=0.284]


Dropout: 0.4, LR: 0.0001, Validation Loss: 0.09374710329142204, Validation Accuracy: 97.30387188323536%
Training complete


Epoch 10/10: 100%|██████████| 1439/1439 [03:27<00:00,  6.95batch/s, loss=0.167]


Dropout: 0.4, LR: 0.0005, Validation Loss: 0.032358311380702, Validation Accuracy: 99.08777620109467%
Training complete


Epoch 10/10: 100%|██████████| 1439/1439 [03:46<00:00,  6.36batch/s, loss=0.177]


...

Dropout: 0.5, LR: 0.001, Validation Loss: 0.07225743699238657, Validation Accuracy: 97.66876140279749%
Training complete


Epoch 10/10: 100%|██████████| 1439/1439 [02:34<00:00,  9.34batch/s, loss=0.483]


Dropout: 0.5, LR: 0.005, Validation Loss: 0.12832137884144162, Validation Accuracy: 96.0571660247314%
Training complete


Epoch 10/10: 100%|██████████| 1439/1439 [03:11<00:00,  7.52batch/s, loss=0.878]


Dropout: 0.5, LR: 0.01, Validation Loss: 0.30431127762813787, Validation Accuracy: 90.22906953172512%
Training complete


In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(all_labels, all_predictions)
cmd = ConfusionMatrixDisplay(cm, display_labels=[i for i in range(num_classes)])
fig, ax = plt.subplots(figsize=(10, 10))  # Adjust the size
cmd.plot(ax=ax, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# debugging for mps speedup